In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
from llama_index.core.schema import TextNode
from typing import List
import json


def get_text_nodes(json_list: List[dict]):
    text_nodes = []
    for idx, page in enumerate(json_list):
        text_node = TextNode(text=page["md"], metadata={"page": page["page"]})
        text_nodes.append(text_node)
    return text_nodes


def save_jsonl(data_list, filename):
    """Save a list of dictionaries as JSON Lines."""
    with open(filename, "w") as file:
        for item in data_list:
            json.dump(item, file)
            file.write("\n")


def load_jsonl(filename):
    """Load a list of dictionaries from JSON Lines."""
    data_list = []
    with open(filename, "r") as file:
        for line in file:
            data_list.append(json.loads(line))
    return data_list

In [ ]:
from llama_parse import LlamaParse

parser = LlamaParse(
    result_type="markdown",
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="anthropic-sonnet-3.5",
    api_key='Your Api key'
)
json_objs = parser.get_json_result(r"F:\Sharry\extraction\2023-conocophillips-aim-presentation_1.pdf")
json_list = json_objs[0]["pages"]
docs = get_text_nodes(json_list)

Started parsing the file under job_id dd4a2a11-be4b-4584-a977-dccb2298423c


In [4]:
# using Sonnet-3.5
print(docs[10].get_content(metadata_mode="all"))

page: 11

# Commitment to Disciplined Reinvestment Rate

Disciplined Reinvestment Rate is the Foundation for Superior Returns on and of Capital, while Driving Durable CFO Growth

| Industry Growth Focus | ConocoPhillips Strategy Reset | Key Metrics |
|----------------------|------------------------------|-------------|
| >100% Reinvestment Rate | <60% Reinvestment Rate | ~50% 10-Year Reinvestment Rate |
| | | ~6% CFO CAGR 2024-2032 |
| | | at $60/BBL WTI Mid-Cycle Planning Price |

| Year Range | ConocoPhillips Average Annual Reinvestment Rate (%) | Notes |
|------------|---------------------------------------------------|-------|
| 2012-2016 | 100% | ~$75/BBL WTI Average, Historic Reinvestment Rate |
| 2017-2022 | ~57% | ~$63/BBL WTI Average, Historic Reinvestment Rate |
| 2023E | ~50% | at $80/BBL WTI, Reinvestment Rate at $60/BBL WTI |
| 2024-2028 | ~55% | at $60/BBL WTI, Reinvestment Rate at $60/BBL WTI |
| | ~42% | Reinvestment Rate at $80/BBL WTI |
| 2029-2032 | ~37% | at $60/BBL

RAG Pipeline

In [5]:
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import Settings


Settings.llm = Ollama(model="llama3.1")
Settings.embed_model = OllamaEmbedding(model_name="nomic-embed-text")

In [6]:
# from llama_index.core import SummaryIndex
from llama_index.core import VectorStoreIndex
from llama_index.llms.ollama import Ollama

index = VectorStoreIndex(docs)
query_engine = index.as_query_engine(similarity_top_k=5)

In [7]:
query = "What is Historic Reinvestment Rate "

response = query_engine.query(query)

In [8]:
print(response)

~100% (for 2012-2016) and ~57% (for 2017-2022)


In [9]:
print(response.source_nodes[3].get_content())

# Significant Free Cash Flow Growth Over the Decade

| Capital ($B) | 2023E | 2024-2028 Average | 2029-2032 Average |
|--------------|-------|-------------------|-------------------|
| Value        | 6.3   | 6.5               | 8.1               |

| Production (MBOED) | 2023E | 2024-2028 Average | 2029-2032 Average |
|--------------------|-------|-------------------|-------------------|
| Value              | 1050  | 1220              | 1530              |

| FCF ($B)           | 2023E | 2024-2028 Average | 2029-2032 Average |
|--------------------|-------|-------------------|-------------------|
| $60/BBL WTI        | 7     | 5.5               | 11                |
| Upside Sensitivity | -     | 3                 | 3.5               |
| Total              | 7     | 8.5               | 14.5              |

## ~$65B FCF and ~50% Reinvestment Rate
Over the Next 10 Years at $60/BBL WTI

*Note: The image includes a map of the United States with some highlighted areas, likely indicating op